This is created by wakuphas on 2019.9.17.

For running YOLO training on GPU with using Google Colab.


# **1. Google Colab**

**あまりないが環境構築等で元に戻したくなったら、以下コマンドで初期化できる。**

In [0]:
!kill -9 -1

**12時間ルールは、下記shコマンドをセルから実行すれば、起動からの時間がわかる。**

12時間すぎると自動でGPUは切断される。数時間経つとばまた使用可能。ただし、アップロードしたファイルや出力されたファイルは全て削除されてしまうので、逐一重みファイルなどの出力ファイルはローカルに保存しておくことをおすすめする。

In [1]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'

0.000741435days (64.06sec)


#** 2. 環境構築**

## **CUDAをインストールする**

In [3]:
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb


Redirecting output to ‘wget-log’.


In [4]:
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

Selecting previously unselected package cuda-repo-ubuntu1604-8-0-local-ga2.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb ...
Unpacking cuda-repo-ubuntu1604-8-0-local-ga2 (8.0.61-1) ...
Setting up cuda-repo-ubuntu1604-8-0-local-ga2 (8.0.61-1) ...
Note: Check first if apt-key functionality is needed at all - it probably isn't!
OK


In [5]:
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub

OK


In [6]:
!apt update

Get:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Ign:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Get:3 file:/var/cuda-repo-8-0-local-ga2  Release.gpg [819 B]
Get:3 file:/var/cuda-repo-8-0-local-ga2  Release.gpg [819 B]
Get:4 file:/var/cuda-repo-8-0-local-ga2  Packages [22.7 kB]
Hit:5 http://security.ubuntu.com/ubuntu artful-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu artful InRelease
Hit:7 http://archive.ubuntu.com/ubuntu artful-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu artful-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
All packages are up to date.


In [7]:
!apt install cuda-8-0 -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  accountsservice acpid activity-log-manager adwaita-icon-theme apg aspell
  aspell-en at-spi2-core avahi-daemon avahi-utils bbswitch-dkms bind9-host
  bluez bluez-obexd bsdmainutils ca-certificates-java cheese-common
  cracklib-runtime crda cuda-command-line-tools-8-0 cuda-core-8-0
  cuda-cublas-8-0 cuda-cublas-dev-8-0 cuda-cudart-8-0 cuda-cudart-dev-8-0
  cuda-cufft-8-0 cuda-cufft-dev-8-0 cuda-curand-8-0 cuda-curand-dev-8-0
  cuda-cusolver-8-0 cuda-cusolver-dev-8-0 cuda-cusparse-8-0
  cuda-cusparse-dev-8-0 cuda-demo-suite-8-0 cuda-documentation-8-0
  cuda-driver-dev-8-0 cuda-drivers cuda-license-8-0 cuda-misc-headers-8-0
  cuda-npp-8-0 cuda-npp-dev-8-0 cuda-nvgraph-8-0 cuda-nvgraph-dev-8-0
  cuda-nvml-dev-8-0 cuda-nvrtc-8-0 cuda-nvrtc-dev-8-0 cuda-runtime-8-0
  cuda-samples-8-0 cuda-toolkit-8-0 cuda-visual-tools-8-0 cups-pk-helper dbus
 

In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

## **gcc-4.8を導入する**
gcc version 4.9以降は対応していないらしい

現在のバージョンを確認

In [9]:
!gcc -dumpversion

7


**gcc-4.8をインストール**

In [10]:
!apt-get install gcc-4.8 g++-4.8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cpp-4.8 gcc-4.8-base libasan0 libgcc-4.8-dev libstdc++-4.8-dev
Suggested packages:
  gcc-4.8-locales g++-4.8-multilib gcc-4.8-doc libstdc++6-4.8-dbg
  gcc-4.8-multilib libgcc1-dbg libgomp1-dbg libitm1-dbg libatomic1-dbg
  libasan0-dbg libtsan0-dbg libquadmath0-dbg libstdc++-4.8-doc
The following NEW packages will be installed:
  cpp-4.8 g++-4.8 gcc-4.8 gcc-4.8-base libasan0 libgcc-4.8-dev
  libstdc++-4.8-dev
0 upgraded, 7 newly installed, 0 to remove and 0 not upgraded.
Need to get 29.3 MB of archives.
After this operation, 73.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/universe amd64 gcc-4.8-base amd64 4.8.5-4ubuntu6 [15.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu artful/universe amd64 cpp-4.8 amd64 4.8.5-4ubuntu6 [4,366 kB]
Get:3 http://archive.ubuntu.com/ubuntu artful/universe amd64 libas

In [11]:
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 40 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8

!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-7 50 --slave /usr/bin/g++ g++ /usr/bin/g++-7

update-alternatives: using /usr/bin/gcc-4.8 to provide /usr/bin/gcc (gcc) in auto mode
update-alternatives: using /usr/bin/gcc-7 to provide /usr/bin/gcc (gcc) in auto mode


## **nvccをインストールする（nvidia-cuda-toolkit）**

In [12]:
!apt install nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libcuinj64-8.0 libcusparse8.0 libnppc8.0 libnppi8.0 libnppial8.0
  libnppicc8.0 libnppicom8.0 libnppidei8.0 libnppif8.0 libnppig8.0 libnppim8.0
  libnppist8.0 libnppisu8.0 libnppitc8.0 libnpps8.0 libnvblas8.0 libnvgraph8.0
  libnvrtc8.0 libnvtoolsext1 libnvvm3 libthrust-dev libvdpau-dev
  nvidia-cuda-dev nvidia-cuda-doc nvidia-cuda-gdb nvidia-opencl-dev
  nvidia-profiler nvidia-visual-profiler opencl-c-headers opencl-clhpp-headers
  opencl-headers
Suggested packages:
  libvdpau-doc libcupti-dev nvidia-driver opencl-clhpp-headers-doc
Recommended packages:
  libnvcuvid1
The following NEW packages will be installed:
  libcuinj64-8.0 libcusparse8.0 libnppc8.0 libnppi8.0 libnppial8.0
  libnppicc8.0 libnppicom8.0 libnppidei8.0 libnppif8.0 libnppig8.0 libnppim8.0
  libnppist8.0 libnppisu8.0 libnppitc8.0 libnpps8.0 libnvblas8.0 libnvgraph8.0
  l

# **3. 実行手順**

## ** Google Driveへdarknetをアップロード**
**学習データは以下を使用**

**https://timebutt.github.io/static/how-to-train-yolov2-to-##detect-custom-objects/**

In [13]:
!ls


cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb  sample_data  wget-log


In [14]:
!pip install PyDrive googledrivedownloader

    100% |████████████████████████████████| 993kB 6.8MB/s 
  Running setup.py bdist_wheel for PyDrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [15]:
from google_drive_downloader import GoogleDriveDownloader

GoogleDriveDownloader.download_file_from_google_drive(file_id='1gaj2Euvjcpv5miElYhMFOJOkm8hXWXaL',
                                                      dest_path="./tmp.zip",
                                                      unzip=True)
!rm tmp.zip        
!ls

Unzipping...Done.
airport						       __MACOSX     wget-log
cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb  sample_data


## **makeを通す**
**Makefileでは GPU=1 を確認**

In [0]:
# opencv = 1 にすると現状動かないが問題ない。

In [3]:
!cat airport/Makefile

GPU=1
CUDNN=0
OPENCV=0
OPENMP=0
DEBUG=0

ARCH= -gencode arch=compute_30,code=sm_30 \
      -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52]
#      -gencode arch=compute_20,code=[sm_20,sm_21] \ This one is deprecated?

# This is what I use, uncomment if you know your arch and want to specify
# ARCH= -gencode arch=compute_52,code=compute_52

VPATH=./src/:./examples
SLIB=libdarknet.so
ALIB=libdarknet.a
EXEC=darknet
OBJDIR=./obj/

CC=gcc
CPP=g++
NVCC=nvcc 
AR=ar
ARFLAGS=rcs
OPTS=-Ofast
LDFLAGS= -lm -pthread 
COMMON= -Iinclude/ -Isrc/
CFLAGS=-Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC

ifeq ($(OPENMP), 1) 
CFLAGS+= -fopenmp
endif

ifeq ($(DEBUG), 1) 
OPTS=-O0 -g
endif

CFLAGS+=$(OPTS)

ifeq ($(OPENCV), 1) 
COMMON+= -DOPENCV
CFLAGS+= -DOPENCV
LDFLAGS+= `pkg-config --libs opencv` -lstdc++
COMMON+= `pkg-config --cflags opencv` 
endif

ifeq ($(GPU), 1) 
COMMON+= -DGPU -I/u

**darknet内でmakeする**

In [4]:
%%bash

cd airport
make

gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/utils.c -o obj/utils.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/cuda.c -o obj/cuda.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/deconvolutional_layer.c -o obj/deconvolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DGPU -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/ -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-

## **darknetを用いて学習させる**
初期値：

yolo v2 のときは darknet19_448.conv.23

yolo v3 のときは darknet53.conv.74

In [0]:
%%bash
cd airport

# v2
./darknet detector train cfg/obj.data cfg/yolo-obj.cfg darknet19_448.conv.23 > train_log.txt
#./darknet detector train cfg/obj.data cfg/yolo-obj.cfg  backup/yolo-obj_600.weights > train_log.txt
#./darknet detector train cfg/airport.data cfg/yolov2-airport.cfg backup/yolov2-airport_1200.weights > train_log.txt

# v3
#./airport detector train cfg/obj.data cfg/yolo-obj.cfg darknet53.conv.74


## **学習したモデルでテスト！**

In [0]:
%%bash
cd airport

# v2
./darknet detector test cfg/yolov2-airport.cfg backup/yolov2-airport_1200.weights test.png
# v3
#./darknet detector test cfg/yolov3-airport.cfg bin/yolov3.weights test.png

names: Using default 'data/names.list'
bash: line 4: 11272 Segmentation fault      (core dumped) ./darknet detector test cfg/yolov2-airport.cfg backup/yolov2-airport_1200.weights test.png
